In [2]:
# !pip install -q tensorflow==1.15.0
# !pip install -q tensorflow-hub
# !pip install -q seaborn

In [4]:
# !pip install bert-tensorflow

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import strings
tf.compat.v1.enable_eager_execution()
# tf.compat.v1.disable_eager_execution()

# !pip install -q tensorflow-hub
# !pip install -q tfds-nightly

import tensorflow_hub as hub
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt 

import time

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  1.15.0
Eager mode:  True
Hub version:  0.7.0
GPU is NOT AVAILABLE


In [2]:
# import bert
# from bert import run_classifier
# from bert import optimization
# from bert import tokenization

In [ ]:
# Set up notebook to work with our large dataset
import resource
low, high = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (2**14, 2**14))

In [10]:
# download, split dataset using TFDS
# we receive the data inside of tensor objects

train_data, validation_data, test_data = tfds.load(name="cnn_dailymail", 
                                                   split=["train", "validation", "test"], 
                                                   batch_size=-1, 
                                                   as_supervised=True)

# #separate into articles and highlights (ex and HL)
# train_ex, train_HL = tfds.as_numpy(train_data)
# val_ex, val_HL = tfds.as_numpy(validation_data)
# test_ex, test_HL = tfds.as_numpy(test_data)

NameError: name 'val_data' is not defined

There are more than 300,000 articles in total. We probably won't use that many, but here we are.

In [12]:
print("Training entries: {}, validation entries: {}, test entries: {}".format(len(train_data[0]), len(validation_data[0]), len(test_data[0])))

Training entries: 287113, validation entries: 13368, test entries: 11490


In [7]:
# mini_train = list(train_data[:5000])
# mini_val = list(validation_data[:5000])

In [8]:
# create mini training/validation datasets for development
# possibly unnecessary for the way we're implementing BERT.
# mini_train_ex, mini_train_HL = train_ex[:5000], train_HL[:5000]
# mini_val_ex, mini_val_HL = val_ex[:5000], val_HL[:5000]
# mini_train_ex[:1], mini_train_HL[:1]

Use .numpy to get data from tensor object, and stringify

In [15]:
# first index chooses highlights or summary, while second index is example number
# sliced_train_articles[0][1] gives article with index 1
# .numpy extracts the tensor value (in bytes)
# we stringify it before adding to pandas

str(train_data[1].numpy()[0]) 

"b'Bishop John Folda, of North Dakota, is taking time off after being diagnosed .\\nHe contracted the infection through contaminated food in Italy .\\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed .'"

### Building Pandas tables and CSVs from our data.
After a few unsuccessful attempts to extract the data from the training set, we discovered that batching the process sped it up enormously. Originally, it took about 1 to 10 seconds to process a single example from the training set. Slicing the training data into sets of 5000 examples allowed us to finish processing the set in seconds.

In [4]:
#287,113 training examples divided into slices of 5000 is 57.4 slices
slices = list(range(0,58))

sliced_train_articles = []
sliced_train_highlights = []

#divide training data into list of lists w/ 5000 tensors/examples each
for ii in slices:
    start = ii*5000
    end = (ii+1)*5000
    sliced_train_articles.append(list(train_data[0][start:end]))
    sliced_train_highlights.append(list(train_data[1][start:end]))

In [5]:
train_articles = []
train_highlights = []

start_time=time.time()

#loop through list of lists
for ii in range(0,58):
    #special case for last list with <5000 examples
    if ii == 57:
        for jj in range(0,2113):
            #append stringified value into corresponding list, article or highlight
            train_articles.append(str(sliced_train_articles[ii][jj].numpy()))
            train_highlights.append(str(sliced_train_highlights[ii][jj].numpy()))   
    else:
        for jj in range(0,5000):
            train_articles.append(str(sliced_train_articles[ii][jj].numpy()))
            train_highlights.append(str(sliced_train_highlights[ii][jj].numpy()))   

print("--- %s seconds ---" % (time.time() - start_time))    

--- 9.266492366790771 seconds ---


We put our data into a Pandas DataFrame for ease of use with different models and examples floating around the internet as notebooks and blog posts. Exporting it as CSV allows us to easily import the data into any project. The data saved as CSV is negligibly larger than when it is stored in other formats.

In [6]:
#zip into list of tuples, prep for pandas
cnn_train = list(zip(train_articles, train_highlights))

In [7]:
#to dataframe
df_train = pd.DataFrame(data=cnn_train, dtype = 'str', columns=['article','highlights'])

In [12]:
#save as csv for posterity
with open('train_dataframe.csv', 'w') as f1:
    df_train.to_csv(f1)

Repeat the process with validation and test data splits

In [9]:
#13,368 validation examples divided into slices of 5000 is 3 slices
slices = list(range(0,3))

sliced_val_articles = []
sliced_val_highlights = []

#divide training data into list of lists w/ 5000 tensors/examples each
for ii in slices:
    start = ii*5000
    end = (ii+1)*5000
    sliced_val_articles.append(list(validation_data[0][start:end]))
    sliced_val_highlights.append(list(validation_data[1][start:end]))
    
    
val_articles = []
val_highlights = []

start_time=time.time()

#loop through list of lists
for ii in range(0,3):
    #special case for last list with <5000 examples
    if ii == 2:
        for jj in range(0,3367):
            #append stringified value into corresponding list, article or highlight
            val_articles.append(str(sliced_val_articles[ii][jj].numpy()))
            val_highlights.append(str(sliced_val_highlights[ii][jj].numpy()))   
    else:
        for jj in range(0,5000):
            val_articles.append(str(sliced_val_articles[ii][jj].numpy()))
            val_highlights.append(str(sliced_val_highlights[ii][jj].numpy()))   

print("--- %s seconds ---" % (time.time() - start_time))    

#zip into list of tuples, prep for pandas
cnn_val = list(zip(val_articles, val_highlights))

#to dataframe
df_val = pd.DataFrame(data=cnn_val, dtype = 'str', columns=['article','highlights'])

#save as csv for posterity
with open('validation_dataframe.csv', 'w') as f1:
    df_val.to_csv(f1)

--- 0.42009592056274414 seconds ---


In [18]:
#11,490 test examples divided into slices of 5000 is 3 slices
slices = list(range(0,3))

sliced_test_articles = []
sliced_test_highlights = []

#divide training data into list of lists w/ 5000 tensors/examples each
for ii in slices:
    start = ii*5000
    end = (ii+1)*5000
    sliced_test_articles.append(list(test_data[0][start:end]))
    sliced_test_highlights.append(list(test_data[1][start:end]))
    
    
test_articles = []
test_highlights = []

start_time=time.time()

#loop through list of lists
for ii in range(0,3):
    #special case for last list with <5000 examples
    if ii == 2:
        for jj in range(0,1489):
            #append stringified value into corresponding list, article or highlight
            test_articles.append(str(sliced_test_articles[ii][jj].numpy()))
            test_highlights.append(str(sliced_test_highlights[ii][jj].numpy()))   
    else:
        for jj in range(0,5000):
            test_articles.append(str(sliced_test_articles[ii][jj].numpy()))
            test_highlights.append(str(sliced_test_highlights[ii][jj].numpy()))   

print("--- %s seconds ---" % (time.time() - start_time))    

#zip into list of tuples, prep for pandas
cnn_test = list(zip(test_articles, test_highlights))

#to dataframe
df_test = pd.DataFrame(data=cnn_test, dtype = 'str', columns=['article','highlights'])

#save as csv for posterity
with open('test_dataframe.csv', 'w') as f1:
    df_test.to_csv(f1)

--- 0.35894203186035156 seconds ---
